In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from inference import fit
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn import tree

from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer

In [2]:
# load data
s = pd.read_csv('diabetes_data.txt',sep= ',', header= None)
s.head()

,0,1,2,3,4,5,6,7,8
0,preg,plas,pres,skin,insu,mass,pedi,age,class
1,6,148,72,35,0,33.6,0.627,50,tested_positive
2,1,85,66,29,0,26.6,0.351,31,tested_negative
3,8,183,64,0,0,23.3,0.672,32,tested_positive
4,1,89,66,23,94,28.1,0.167,21,tested_negative


In [3]:
# select features and target:
s = np.array(s)

# features:
X = s[1:,:8].astype(float)
l,n = X.shape

# target:
y = s[1:,8]
# convert target to number
letter2number = {'tested_negative':-1,'tested_positive':1}
y = np.array([letter2number[y[t]] for t in range(l)])

In [4]:
# impute missing data
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=0, strategy='median')
imp_mean.fit(X)
X = imp_mean.transform(X)

In [5]:
def predict(X,y,kf=5):    
    #x_train,x_test,y_train,y_test = train_test_split(X1,y,test_size=0.3,random_state = 100)    
    kfold = KFold(n_splits=kf,shuffle=True,random_state=1)
    accuracy = np.zeros(kf)
    
    for i,(train_index,test_index) in enumerate(kfold.split(y)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # predict with ER
        h0,w = fit(X_train,y_train,regu=0.005)
        h_pred = h0 + X_test.dot(w)
        y_pred = np.sign(h_pred)
        accuracy[i] = accuracy_score(y_test,y_pred)
        #print(accuracy[i])
    return accuracy.mean(),accuracy.std()

In [6]:
kf = 5
accuracy_mean,accuracy_std = predict(X,y,kf)
print(accuracy_mean,accuracy_std)

(0.7708174178762415, 0.021347553416449002)


In [7]:
# normalize features
from sklearn.preprocessing import maxabs_scale
X1 = maxabs_scale(X)
accuracy_mean,accuracy_std = predict(X1,y,kf)
print(accuracy_mean,accuracy_std)

(0.7656311009252186, 0.014740255090555932)


In [8]:
"""
from sklearn.preprocessing import Normalizer

transformer = Normalizer().fit(X)
Normalizer(copy=True, norm='l2')
X1 = transformer.transform(X)
predict(X1,y)
"""

from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer().fit(X)
PowerTransformer(copy=True, method='yeo-johnson', standardize=True)
X1 = pt.transform(X)
accuracy_mean,accuracy_std = predict(X1,y,kf)
print(accuracy_mean,accuracy_std)

(0.7682369917664035, 0.01816637665292534)
